#### GIS 5571: ArcGIS I
#### Lab 3 - Part 1
#### Mattie Gisselbeck
#### Date Created: November 2, 2022
#### Date Last Updated: December 2, 2022

#### Part 1.1: Import Packages and Request Data from Minnesota Geospatial Commons

In [1]:
# Import Packages
import arcpy
import requests

In [2]:
# Define Variables 
counties = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip"
streams = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_pca/env_assessed_streams_2012/shp_env_assessed_streams_2012.zip"
landcover = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip"
dem = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip"
roads = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_roads_mndot_tis/shp_trans_roads_mndot_tis.zip"

In [3]:
# Request and Extract County Boundaries
c = requests.get(counties)
with open (r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\shp_bdry_counties_in_minnesota.zip', 'wb') as f:
    f.write(c.content)

with zipfile.ZipFile("shp_bdry_counties_in_minnesota.zip", "r") as zip_ref:
    zip_ref.extractall(r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2')
    

# Request and Extract Streams with Strahler Stream Order
s = requests.get(streams)
with open (r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\shp_env_assessed_stremas_2012.zip', 'wb') as f:
    f.write(s.content)

with zipfile.ZipFile("shp_env_assessed_stremas_2012.zip", "r") as zip_ref:
    zip_ref.extractall(r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2')
    

# Request NLCD 
l = requests.get(landcover)
with open (r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\tif_biota_landcover_nlcd_mn_2019.zip', 'wb') as f:
    f.write(l.content
            
with zipfile.ZipFile("tif_biota_landcover_nlcd_mn_2019.zip", "r") as zip_ref:
    zip_ref.extractall(r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2')       


# Request and Extract DEM
d = requests.get(dem)
with open (r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\fgdb_elevation_30m_digital_elevation_model.zip', 'wb') as f:
    f.write(d.content)
    
with zipfile.ZipFile("fgdb_elevation_30m_digital_elevation_model.zip", "r") as zip_ref:
    zip_ref.extractall(r'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2')

#### Part 2.2: Create a Study Extent 

In [5]:
# Create Study Extent (Feature Class to Feature Class)
arcpy.conversion.FeatureClassToFeatureClass("mn_county_boundaries", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb", "StudyExtent", "CTY_NAME = 'Wabasha' Or CTY_NAME = 'Winona' Or CTY_NAME = 'Olmsted'", 'AREA "AREA" true true false 19 Double 0 0,First,#,mn_county_boundaries,AREA,-1,-1;PERIMETER "PERIMETER" true true false 19 Double 0 0,First,#,mn_county_boundaries,PERIMETER,-1,-1;CTYONLY_ "CTYONLY_" true true false 19 Double 0 0,First,#,mn_county_boundaries,CTYONLY_,-1,-1;CTYONLY_ID "CTYONLY_ID" true true false 19 Double 0 0,First,#,mn_county_boundaries,CTYONLY_ID,-1,-1;COUN "COUN" true true false 4 Short 0 4,First,#,mn_county_boundaries,COUN,-1,-1;CTY_NAME "CTY_NAME" true true false 20 Text 0 0,First,#,mn_county_boundaries,CTY_NAME,0,20;CTY_ABBR "CTY_ABBR" true true false 4 Text 0 0,First,#,mn_county_boundaries,CTY_ABBR,0,4;CTY_FIPS "CTY_FIPS" true true false 4 Short 0 4,First,#,mn_county_boundaries,CTY_FIPS,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,mn_county_boundaries,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,mn_county_boundaries,Shape_Area,-1,-1', '')

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\StudyExtent'>

In [6]:
# Dissolve County Boundaries 
arcpy.management.Dissolve("StudyExtent", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\StudyExtent_Dissolve", None, None, "MULTI_PART", "UNSPLIT_LINES")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\StudyExtent_Dissolve'>

#### Part 1.3: Roadway Routes 

In [7]:
# Extract by Mask
Extract_Roads = arcpy.sa.ExtractByMask("NLCD_2019_Land_Cover_Impervious_Descriptor.tif", "StudyExtent_Dissolve"); Extract_Roads.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Extract_Roads")

In [8]:
# Reclassify Roads (Scale: 1-10) - (1: Primary, Secondary, Teriary) (2: Non-road Non-energy Impervious) (3: LCMAP Impervious) (7: Wind Turbines)
arcpy.ddd.Reclassify("Extract_Roads", "Class_Name", "Unclassified 10;'Primary road' 1;'Secondary road' 1;'Tertiary road' 1;'Non-road non-energy impervious' 2;'LCMAP impervious' 3;'Wind turbines' 7", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_Roads", "DATA")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\Reclass_Roads'>

#### Part 1.4: Digital Elevation Model (DEM)

In [9]:
# Extract by Mask
Extract_DEM = arcpy.sa.ExtractByMask(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m", "StudyExtent_Dissolve"); Extract_DEM.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Extract_DEM")

In [2]:
# Calculate Slope
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb", workspace=r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb"):
    arcpy.ddd.Slope("Extract_DEM", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\DEM_Slope", "PERCENT_RISE", 1, "PLANAR", "METER")

In [9]:
# Reclassify Slope (Scale: 1-10)
arcpy.ddd.Reclassify("DEM_Slope", "VALUE", "0 3 1;3 6 2;6 10 3;10 15 4;15 20 8;20 25 9;25 30 10;30 40 10;40 60 10;60 100 10;100 1000 10", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_DEM_2", "DATA")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\Reclass_DEM_2'>

### NLCD

In [13]:
# Extract by Mask 
Extract_NLCD = arcpy.sa.ExtractByMask(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\tif_biota_landcover_nlcd_mn_2019\NLCD_2019_Land_Cover.tif", "StudyExtent_Dissolve"); Extract_NLCD.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Extract_NLCD")

In [14]:
# Reclassify NLCD
arcpy.ddd.Reclassify("Extract_NLCD", "NLCD_Land", "'Open Water' 10;'Developed, Open Space' 2;'Developed, Low Intensity' 2;'Developed, Medium Intensity' 2;'Developed, High Intensity' 2;'Barren Land' 5;'Deciduous Forest' 7;'Evergreen Forest' 7;'Mixed Forest' 7;Shrub/Scrub 7;Herbaceous 7;Hay/Pasture 9;'Cultivated Crops' 9;'Woody Wetlands' 10;'Emergent Herbaceous Wetlands' 10", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_NLCD", "DATA")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\Reclass_NLCD'>

### Streams with Strahler Stream Order

In [17]:
# Clip Streams
arcpy.analysis.Clip("streams_with_strahler_stream_order", "StudyExtent_Dissolve", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Streams_Clip", None)

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\Streams_Clip'>

In [19]:
# Feature to Raster 
arcpy.conversion.FeatureToRaster("Streams_Clip", "SO_VALUE", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Feature_Stre1", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_DEM_Slope")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\Feature_Stre1'>

In [20]:
# Reclassify Streams
arcpy.ddd.Reclassify("Feature_Stre1", "Value", "1 1;2 2;3 3;4 7;5 8;6 9;8 10", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_Streams", "DATA")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\Reclass_Streams'>

In [21]:
# Raster Calculator (Note: Command only worked in Geoprocessing pane in ArcGIS Pro)
output_raster = arcpy.ia.RasterCalculator(' Con(IsNull("Reclass_Streams"),1, "Reclass_Streams")'); output_raster.save(r"c:\Users\gisse015\documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\RC_Streams")

TypeError: RasterCalculator() missing 2 required positional arguments: 'input_names' and 'expression'

### Part 2.7: Dory's Farm (Start Point) and Picnic Area (End Point)

In [20]:
# Dory's Farm (Start Point)
arcpy.management.XYTableToPoint("Start.csv", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Start_Point", "LONG", "LAT", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\Start_Point'>

In [21]:
# Picnic Area (End Point)
arcpy.management.XYTableToPoint("End.csv", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\End_Point", "LONG", "LAT", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\End_Point'>

In [25]:
# Point to Raster - Dory's Farm (Start Point)
arcpy.conversion.PointToRaster("Start_Point", "OBJECTID", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Start_RasterPoint", "MOST_FREQUENT", "NONE", None, "BUILD")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\Start_RasterPoint'>

In [26]:
# Point to Raster - Picnic Area (End Point)
arcpy.conversion.PointToRaster("End_Point", "OBJECTID", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\End_RasterPoint", "MOST_FREQUENT", "NONE", None, "BUILD")

<Result 'C:\\Users\\gisse015\\Documents\\ArcGIS\\Projects\\Lab2_2\\Lab2_2.gdb\\End_RasterPoint'>

### Part 2.8: Weighted Overlay (Cost Surface)

In [22]:
# Weighted Overlay (Cost Surface)
Cost_Surface = arcpy.sa.WeightedOverlay("('Raster_C' 25 'Value' (1 1; 2 2; 3 3; 7 8; 8 9; 9 10; 10 10; NODATA NODATA); 'Reclass_DEM_Slope' 25 'Value' (2 2; 3 3; 4 4; 7 8; 8 9; 10 10; NODATA NODATA); 'Reclass_NLCD' 25 'Value' (2 2; 5 6; 7 8; 9 9; 10 10; NODATA NODATA); 'Reclass_Roads' 25 'Value' (1 1; 2 2; 3 3; 7 8; 10 10; NODATA NODATA));1 10 1"); Cost_Surface.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Weighte_Rast1")

### Part 2.9: Cost Distance and Cost Back Link

In [ ]:
# Cost Distance
Cost_Distance = arcpy.sa.CostDistance("DorysFarm", "out_raster", None, r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Direction_Raster", None, None, None, None, ''); Cost_Distance.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\CostDis")

### Part 2.10: Cost Path (Optimal Route for Dory)

In [1]:
# Cost Path (Best Single) - Optimal Route for Dory
Cost_Path = arcpy.sa.CostPath("PicnicArea", "CostDis", "Direction_Raster", "BEST_SINGLE", "ID", "INPUT_RANGE"); Cost_Path.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\CostPath")

## Lab 3 - Part 1: Comparing Weighting Approaches for Optimizing Cost Surface

### Part 1.1: Cost Path B

In [ ]:
# Reclassify DEM Slope
arcpy.ddd.Reclassify("DEM_Slope", "VALUE", "0 12.552466 1;12.552466 31.381165 1;31.381165 54.394019 2;54.394019 81.591028 2;81.591028 108.788037 5;108.788037 133.892969 10;133.892969 156.905823 10;156.905823 179.918677 10;179.918677 209.207764 10;209.207764 263.601782 10;263.601782 533.479797 11", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_DEM_2_OP", "DATA")

# Reclassify NLCD Land Cover
arcpy.ddd.Reclassify("Extract_NLCD", "NLCD_Land", "'Open Water' 10;'Developed, Open Space' 2;'Developed, Low Intensity' 4;'Developed, Medium Intensity' 4;'Developed, High Intensity' 5;'Barren Land' 2;'Deciduous Forest' 6;'Evergreen Forest' 6;'Mixed Forest' 9;Shrub/Scrub 4;Herbaceous 5;Hay/Pasture 10;'Cultivated Crops' 10;'Woody Wetlands' 7;'Emergent Herbaceous Wetlands' 8", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_NLCD_OP_2", "DATA")

# Cost Surface
out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_NLCD_OP_2' 25 'Value' (2 2; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 10; NODATA NODATA); 'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_DEM_2_OP' 25 'Value' (1 1; 2 2; 5 5; 10 10; 11 10; NODATA NODATA); 'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Raster_C' 25 'Value' (1 1; 2 2; 3 3; 7 7; 8 8; 9 9; 10 10; NODATA NODATA); 'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_Roads' 25 'Value' (1 1; 2 2; 3 3; 7 7; 10 10; NODATA NODATA));1 10 1"); out_raster.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Weighte_Recl_2")

# Cost Distance 
out_distance_raster = arcpy.sa.CostDistance("DorysFarm", "Weighte_Recl_2", None, r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Backlink_OP_2", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\CostDis_OP_2")

# Cost Path
out_raster = arcpy.sa.CostPath("PicnicArea", "CostDis_OP_2", "Backlink_OP_2", "EACH_CELL", "ID", "INPUT_RANGE"); out_raster.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\CostPat_Picn2")

#### Part 1.2: Cost Path C

In [ ]:
# Reclassify NLCD Land Cover
arcpy.ddd.Reclassify("Extract_NLCD", "NLCD_Land", "'Open Water' 10;'Developed, Open Space' 3;'Developed, Low Intensity' 3;'Developed, Medium Intensity' 3;'Developed, High Intensity' 3;'Barren Land' 4;'Deciduous Forest' 5;'Evergreen Forest' 5;'Mixed Forest' 6;Shrub/Scrub 5;Herbaceous 5;Hay/Pasture 10;'Cultivated Crops' 10;'Woody Wetlands' 10;'Emergent Herbaceous Wetlands' 15", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_OP_3", "DATA")

# Cost Surface
out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_OP_3' 34 'Value' (3 3; 4 4; 5 5; 6 6; 10 10; 15 10; NODATA NODATA); 'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Raster_C' 33 'Value' (1 1; 2 2; 3 3; 7 8; 8 9; 9 10; 10 10; NODATA NODATA); 'C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Reclass_Roads' 33 'Value' (1 1; 2 2; 3 3; 7 7; 10 10; NODATA NODATA));1 10 1"); out_raster.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Weighte_Recl5")

# Cost Distance
out_distance_raster = arcpy.sa.CostDistance("DorysFarm", "Weighte_Recl5", None, r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\BL_5", None, None, None, None, ''); out_distance_raster.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\CostDis_Dory8")

# Cost Path
out_raster = arcpy.sa.CostPath("PicnicArea", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\CostDis_Dory8", r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\BL_5", "EACH_CELL", "ID", "INPUT_RANGE"); out_raster.save(r"C:\Users\gisse015\Documents\ArcGIS\Projects\Lab2_2\Lab2_2.gdb\Cost_Path_3")